In [11]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage
from typing import Annotated

In [6]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

api_key = os.getenv("QWEN_API_KEY")
base_url = os.getenv("QWEN_API_BASE_URL")

llm = ChatOpenAI(model="qwen-plus-latest",api_key=api_key,base_url=base_url,temperature=0.01)

In [7]:
class OverallState(TypedDict):
    messages: Annotated[list,add_messages]
    result: str

In [9]:
async def node1(state:OverallState)->OverallState:
    response = await llm.ainvoke(state["messages"])
    return {"messages":[response]}

In [10]:
graph_builder = StateGraph(OverallState)

graph_builder.add_node("node1",node1)

graph_builder.add_edge(START, "node1")
graph_builder.add_edge("node1",END)

graph = graph_builder.compile()

In [12]:
result = await graph.ainvoke({'messages':[HumanMessage('who are you?')]})

In [13]:
result

{'messages': [HumanMessage(content='who are you?', additional_kwargs={}, response_metadata={}, id='1e5012ab-90c7-4df6-801f-a53d68884007'),
  AIMessage(content='I am a large-scale language model independently developed by the Tongyi Lab under Alibaba Group. My name is Qwen. I am capable of answering questions, creating text such as stories, official documents, emails, scripts, and more. I can also perform logical reasoning, coding, express opinions, play games, and assist with various other tasks. How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 16, 'total_tokens': 92, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-plus-latest', 'system_fingerprint': None, 'id': 'chatcmpl-78874fc1-84f6-9d11-9557-adf677806b43', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--f11f2c17-c256-45d2-8120-b39dc3ff0dfb-0', usage_metadata={'input_tokens': 

In [17]:
# default stream_mode is updates
async for chunk in graph.astream({'messages':[HumanMessage('who are you?')]}, stream_mode="updates"):
    print(chunk)

{'node1': {'messages': [AIMessage(content='I am a large-scale language model independently developed by the Tongyi Lab under Alibaba Group. My name is Qwen. I am capable of answering questions, creating text such as stories, official documents, emails, scripts, and more. I can also perform logical reasoning, coding, express opinions, play games, and handle various other tasks. If you have any questions or need assistance, feel free to ask me anytime!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 16, 'total_tokens': 100, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-plus-latest', 'system_fingerprint': None, 'id': 'chatcmpl-33c2688b-c54e-93be-ad27-cb96fc387cd8', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--1be5f643-041b-4620-a29c-2d3d3bd4f5f4-0', usage_metadata={'input_tokens': 16, 'output_tokens': 84, 'total_tokens': 100, 'input_token_details': {},

In [19]:
async for chunk in graph.astream({'messages':[HumanMessage('who are you?')]}, stream_mode="values"):
    print(chunk)

{'messages': [HumanMessage(content='who are you?', additional_kwargs={}, response_metadata={}, id='293ccf77-ae58-4b05-aac4-27158d9d5e45')]}
{'messages': [HumanMessage(content='who are you?', additional_kwargs={}, response_metadata={}, id='293ccf77-ae58-4b05-aac4-27158d9d5e45'), AIMessage(content='I am a large-scale language model independently developed by the Tongyi Lab under Alibaba Group. My name is Qwen. I am capable of answering questions, creating text such as stories, official documents, emails, scripts, and more. I can also perform logical reasoning, coding, express opinions, play games, and so on. I support multiple languages, including Chinese, English, German, French, Spanish, Portuguese, Italian, Dutch, Russian, Czech, Polish, Arabic, Persian, Hebrew, Turkish, Japanese, Korean, and many others. If you have any questions or need assistance, feel free to let me know anytime!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 127, 'p

In [20]:
async for chunk in graph.astream({'messages':[HumanMessage('who are you?')]}, stream_mode="messages"):
    print(chunk)

(AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run--cb669bfb-8c06-4b64-b132-cdcfc887a876'), {'langgraph_step': 1, 'langgraph_node': 'node1', 'langgraph_triggers': ('branch:to:node1',), 'langgraph_path': ('__pregel_pull', 'node1'), 'langgraph_checkpoint_ns': 'node1:9a2a84ba-8750-e0fd-ca60-cd5765562534', 'checkpoint_ns': 'node1:9a2a84ba-8750-e0fd-ca60-cd5765562534', 'ls_provider': 'openai', 'ls_model_name': 'qwen-plus-latest', 'ls_model_type': 'chat', 'ls_temperature': 0.01})
(AIMessageChunk(content='I', additional_kwargs={}, response_metadata={}, id='run--cb669bfb-8c06-4b64-b132-cdcfc887a876'), {'langgraph_step': 1, 'langgraph_node': 'node1', 'langgraph_triggers': ('branch:to:node1',), 'langgraph_path': ('__pregel_pull', 'node1'), 'langgraph_checkpoint_ns': 'node1:9a2a84ba-8750-e0fd-ca60-cd5765562534', 'checkpoint_ns': 'node1:9a2a84ba-8750-e0fd-ca60-cd5765562534', 'ls_provider': 'openai', 'ls_model_name': 'qwen-plus-latest', 'ls_model_type': 'chat', 'ls_temp